In [18]:
import pygame
import math
pygame.init()#initalize pygame

(5, 0)

In [19]:
font = pygame.font.SysFont("Arial",16)
title_font = pygame.font.SysFont('Arial', 24, bold=True)
planet_name = pygame.font.SysFont('Arial',16)

In [20]:
YELLOW = (255,255,0)
BLUE = (0,32,255)
L_BLUE = (80, 208, 255)
ORANGE = (255,160,16)
L_ORANGE =(255,208,160)
BROWN = (160,128,60)
RED = (255,0,0)
BLACK = (0,0,0)
DARK_GREY = (100,100,100)
WHITE = (255,255,255)

In [21]:
class Planet :
   AU = 149.6e6 *1000 #metre
   G = 6.67428e-11
   SCALE = 10/AU
   TIMESTEP = 3600*24 #1 day
   zoom = 1.0
   ZOOM_STEP = 0.1
   MIN_ZOOM = 0.1
   MAX_ZOOM = 10.0

   def __init__(self,name,x,y,radius,color,mass) :
       self.x = x
       self.y = y
       self.radius = radius
       self.mass = mass
       self.color = color
       self.x_vel = 0
       self.y_vel = 0
       self.sun = False
       self.distance_to_sun = 0
       self.orbit = []
       self.name = name
       
   def draw(self,win,zoom) :
       scale = self.SCALE * zoom
       x =self.x*scale + (WIDTH/2)
       y =self.y*scale + (HEIGHT/2)

       updated_points =[]
       if len(self.orbit) > 2 :
          for point in self.orbit :
              x , y = point 
              x =x*scale + (WIDTH/2)
              y =y*scale + (HEIGHT/2)
              updated_points.append((x,y))
          pygame.draw.lines(win,self.color,False,updated_points)
       
       pygame.draw.circle(win,self.color,(x,y),max(1,int(self.radius*zoom)))
           
       if  not self.sun :
           distance_text = font.render(f'{round(self.distance_to_sun/1000,1)}km',1,WHITE)
           win.blit(distance_text,(x-distance_text.get_width()/2,y-distance_text.get_width()/2))
           title = title_font.render("Solar System", True, WHITE)
           win.blit(title, (WIDTH - 180, 20))
           planet = planet_name.render(self.name,1, WHITE)
           win.blit(planet,  (x + 5, y + 5))

       
   def attraction (self,other) :
       other_x,other_y = other.x,other.y
       distance_x  = other_x-self.x
       distance_y = other_y-self.y
       distance = math.sqrt (distance_x**2 + distance_y**2)
       if other.sun :
           self.distance_to_sun = distance 
       force =( Planet.G * self.mass*other.mass ) / distance**2
       theta = math.atan2(distance_y,distance_x)
       force_x = force * math.cos(theta)
       force_y = force* math.sin(theta)
       return force_x,force_y

   def update_positions(self,planets) :
        total_fx = total_fy = 0
        for planet in planets :
            if self == planet :
                continue 
            fx ,fy = self.attraction(planet)
            total_fx += fx
            total_fy += fy
        self.x_vel += total_fx / self.mass * self.TIMESTEP
        self.y_vel += total_fy / self.mass * self.TIMESTEP
        self.x += self.x_vel * self.TIMESTEP
        self.y += self.y_vel * self.TIMESTEP
        self.orbit.append((self.x,self.y))

In [22]:
WIDTH = 400
HEIGHT = 400
WIN = pygame.display.set_mode((WIDTH,HEIGHT))
pygame.display.set_caption("Planet Simulations")

def main() :
    zoom = 1.0
    ZOOM_STEP = 0.1
    MIN_ZOOM = 0.2
    MAX_ZOOM = 5.0
    run = True
    clock = pygame.time.Clock()    #controlling the simulation speed 
    sun = Planet("Sun",0,0,2,YELLOW,1.989*10**30)
    
    earth = Planet ("Earth",-1 *Planet.AU ,0,2,BLUE,5.97*10**24)
    earth.y_vel = 29.783*1000
    
    mars = Planet ("Mars",-1.524* Planet.AU ,0 , 2,RED,6.42*10**23)
    mars.y_vel = 24.077*1000
    
    mercury = Planet ("Mercury",-0.39* Planet.AU ,0 , 2,DARK_GREY,3.30*10**23)
    mercury.y_vel = -47.4*1000
    
    venus = Planet ("Venus",-0.72* Planet.AU ,0 , 2,ORANGE,4.87*10**24)
    venus.y_vel = -35.02*1000
    
    neptune = Planet ("Neptune",-30.05* Planet.AU ,0 , 11,L_BLUE,1.02*10**26)
    neptune.y_vel = -5.4*1000

    uranus = Planet ("Uranus",-19.2* Planet.AU ,0 , 12,L_BLUE,8.68*10**25)
    uranus.y_vel = -6.8*1000

    jupiter = Planet ("Jupiter",-5.2* Planet.AU ,0 , 15,L_ORANGE,1.90*10**27)
    jupiter.y_vel = -13.1*1000

    saturn = Planet ("Saturn",-9.58* Planet.AU ,0 , 18,BROWN,5.68*10**26)
    saturn.y_vel = -9.7*1000

    
    sun.sun = True
    planets = [sun,earth,mars,mercury,venus,jupiter,saturn,neptune,uranus]
    
    while run:
      WIN.fill((0,0,0))
      clock.tick(60)    #controlling the simulation speed 
      for event in pygame.event.get():
         if event.type == pygame.QUIT:
           run = False
             
         if event.type == pygame.MOUSEWHEEL:
             if event.y > 0:
                zoom *= (1 + ZOOM_STEP)
             elif event.y < 0:
                zoom *= (1 - ZOOM_STEP)
             zoom = max(MIN_ZOOM, min(MAX_ZOOM, zoom))
      
      for planet in planets :
            planet.update_positions(planets)
            planet.draw(WIN,zoom)
      pygame.display.update()
    pygame.quit()



main()